In [1]:
import time
import os
import sys
from IPython.display import display, clear_output, HTML

import imageio
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from matplotlib import animation, rc
animation.rcParams['animation.writer']='ffmpeg'

from mcClass import *
%matplotlib inline

In [42]:
directory='dyntex_examples/'

filename='goldenline.mp4'
#filename='clouds2.mp4'
#filename='fire2.mp4'
#filename='fountain.mp4'
#filename='mc.mp4'

# init MC
MC = motionCloud(directory=directory, fileName=filename, \
                 overSamp=2, framePerSecond=25, N=64, chooseDev=0, stdConst=35.0, show=1)
# read texture example
MC.read()
MC.N = MC.mov.shape[0]
# compute the PCA color space and transform 
MC.colorPCA(0)
MC.color2pca()

# compute the periodic component 
MC.perComp()

# learn the sPDE parameters alpha, beta and C on the principal color component
MC.learnKernel(MC.movPer[:,:,0,:], 1.0/25.0)

# perform synthesis 
MC.initGPU()
MC.synTex(MC.mov.shape[3])

# transform the synthesis back to the rgb color space 
MC.pca2color(syn=1)
MC.pca2color(syn=0)

In [5]:
# First set up the figure, the axis, and the plot element we want to animate
fig01, axes01 = plt.subplots(nrows=1, ncols=2,figsize=(10, 5))
plt.close()

def init():
    return ()


def animate(i): 
    
    im0 = axes01[0].imshow(np.uint8(MC.mov[:,:,:,i]))
    axes01[0].xaxis.set_visible(False)
    axes01[0].yaxis.set_visible(False)
    
    im0 = axes01[1].imshow(np.uint8(MC.movSyn[:,:,:,i]))
    axes01[1].xaxis.set_visible(False)
    axes01[1].yaxis.set_visible(False)
    
    clear_output(wait=True)
    print 'It: %i'%i
    sys.stdout.flush()
    return (im0,)
            
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig01, animate, init_func=init, frames=MC.mov.shape[3], interval=40, blit=True)
HTML(anim.to_html5_video()) 


It: 99


In [44]:
wvid = imageio.get_writer('goldenline.mp4', fps=25.0)

for i in range(MC.movSyn.shape[3]):
    wvid.append_data(np.uint8(MC.mov[:,:,:,i]))
    
wvid.close()